In [23]:
import requests as requests
import json
import datetime
from datetime import date
import pandas as pd

keys={}
with open("/User/jgluck/Documents/GitHub/new-artists-twitter-bot/keys.json","r") as f:
    keys = json.loads(f.read())
    
# Consumer keys and access tokens, used for OAuth
songkick_api_key = keys['songkick_api_key']
dc_metro_id = "1409"

In [67]:
def getArtistId(artist):
    url = "https://api.songkick.com/api/3.0/search/artists.json"
    params = {'apikey': songkick_api_key, "query" : artist}
    resp = requests.get(url, params=params) 
    return resp.text

def getUpcomingShowsPage(metroId, minDate,MaxDate,page):
    url = "https://api.songkick.com/api/3.0/metro_areas/" + metroId + "/calendar.json"
    params = {'apikey': songkick_api_key, "min_date" : minDate, "max_date" : maxDate, "page" : page, "per_page" : 50}
    resp = requests.get(url, params=params) 
    return json.loads(resp.text)

def getUpcomingShows(daysAhead, metroId):
    notFound = 0
    notFoundArtists = []
    
    artistIds = []
    artistMbids = []
    artistNames = []
    artistUrls = []
    billings = []
    eventIds = []
    eventTypes = []
    eventUrls = []
    dates = []
    venueIds = []
    venueNames = []
    locationCities = []
    
    page = 1
    minDate = str(date.today())
    maxDate = str(date.today() + datetime.timedelta(daysAhead))
    resp = getUpcomingShowsPage(metroId, minDate,maxDate, page)
    results = resp["resultsPage"]["results"]
    
    while (results != {}):
        events = results["event"]
    
        for event in events:
            for performance in event["performance"]:
                artist = performance["artist"]

                print(artist)
                billings.append(performance["billing"])
                artistIds.append(artist["id"])
                artistNames.append(artist["displayName"])
                artistUrls.append(artist["uri"])
                eventIds.append(event['id']) 
                eventTypes.append(event['type'])
                eventUrls.append(event['uri'])
                dates.append(event['start']['date'])
                venueIds.append(event['venue']['id'])
                venueNames.append(event['venue']['displayName'])
                locationCities.append(event['location']['city']) 

         
        page = page + 1
        resp = getUpcomingShowsPage(metroId, minDate,maxDate, page)
        results = resp["resultsPage"]["results"]
    
    data = pd.DataFrame(
            {
                "artistId" : artistIds,
                "artistMbid" : artistMbids,
                "artistName" : artistNames,
                "artistUrl" : artistUrls,
                "billing" : billings,
                "eventId" : eventIds,
                "eventType" : eventTypes,
                "eventUrl" : eventUrls,
                "date" : dates,
                "venueId" : venueIds,
                "venueName" : venueNames,
                "locationCity" : locationCities
            })
    
    print("Artists not in data " + str(notFound))
    print(notFoundArtists)
    
    return data

In [68]:
data = getUpcomingShows(7,dc_metro_id)

{'id': 27001, 'displayName': 'North Mississippi Allstars', 'uri': 'http://www.songkick.com/artists/27001-north-mississippi-allstars?utm_source=59162&utm_medium=partner', 'identifier': [{'mbid': '62fa3eb2-1b73-4029-ba35-16ab66d29d02', 'href': 'http://api.songkick.com/api/3.0/artists/mbid:62fa3eb2-1b73-4029-ba35-16ab66d29d02.json'}]}
{'id': 9278794, 'displayName': 'Black Midi', 'uri': 'http://www.songkick.com/artists/9278794-black-midi?utm_source=59162&utm_medium=partner', 'identifier': [{'mbid': '4afbef0a-05ee-4551-b0e0-f1f1870d8f1c', 'href': 'http://api.songkick.com/api/3.0/artists/mbid:4afbef0a-05ee-4551-b0e0-f1f1870d8f1c.json'}]}
{'id': 304340, 'displayName': 'Fat Tony', 'uri': 'http://www.songkick.com/artists/304340-fat-tony?utm_source=59162&utm_medium=partner', 'identifier': [{'mbid': '01bd8950-11c7-47e8-a3e2-3f662a48752a', 'href': 'http://api.songkick.com/api/3.0/artists/mbid:01bd8950-11c7-47e8-a3e2-3f662a48752a.json'}]}
{'id': 517398, 'displayName': 'Edwin McCain', 'uri': 'http:/

{'id': 439923, 'displayName': 'Frankie Valli', 'uri': 'http://www.songkick.com/artists/439923-frankie-valli?utm_source=59162&utm_medium=partner', 'identifier': [{'mbid': '3c714102-94d8-42f4-9005-fb8a75c2766d', 'href': 'http://api.songkick.com/api/3.0/artists/mbid:3c714102-94d8-42f4-9005-fb8a75c2766d.json'}]}
{'id': 5312223, 'displayName': 'Chastity Belt', 'uri': 'http://www.songkick.com/artists/5312223-chastity-belt?utm_source=59162&utm_medium=partner', 'identifier': [{'mbid': '944267d5-c0f0-4f4e-b3f2-7987d25d0441', 'href': 'http://api.songkick.com/api/3.0/artists/mbid:944267d5-c0f0-4f4e-b3f2-7987d25d0441.json'}]}
{'id': 477615, 'displayName': 'The Risk', 'uri': 'http://www.songkick.com/artists/477615-risk?utm_source=59162&utm_medium=partner', 'identifier': [{'mbid': 'e2ac757f-787a-478d-ba90-96b552c3d928', 'href': 'http://api.songkick.com/api/3.0/artists/mbid:e2ac757f-787a-478d-ba90-96b552c3d928.json'}, {'mbid': 'a9b84962-0a17-48cb-b7da-06e8d46a5216', 'href': 'http://api.songkick.com/a

KeyboardInterrupt: 

In [54]:
print(len(data))

242


In [33]:
data.head(20)

,eventId,eventType,eventUrl,date,venueId,venueName,locationCity
0,38939734,Concert,http://www.songkick.com/concerts/38939734-nort...,2019-11-13,1611908.0,The Hamilton Live,"Washington, DC, US"
1,39006317,Concert,http://www.songkick.com/concerts/39006317-blac...,2019-11-13,839721.0,U Street Music Hall,"Washington, DC, US"
2,39013101,Concert,http://www.songkick.com/concerts/39013101-edwi...,2019-11-13,3719779.0,City Winery,"Washington, DC, US"
3,39036547,Concert,http://www.songkick.com/concerts/39036547-gibs...,2019-11-13,32723.0,Birchmere,"Alexandria, VA, US"
4,39045046,Concert,http://www.songkick.com/concerts/39045046-lisa...,2019-11-13,4100939.0,Songbyrd Vinyl Lounge,"Washington, DC, US"
5,39051092,Concert,http://www.songkick.com/concerts/39051092-pivo...,2019-11-13,3578714.0,Milkboy Arthouse,"College Park, MD, US"
6,39062096,Concert,http://www.songkick.com/concerts/39062096-ra-r...,2019-11-13,922.0,9:30 Club,"Washington, DC, US"
7,39070760,Concert,http://www.songkick.com/concerts/39070760-pete...,2019-11-13,124690.0,Jammin' Java,"Vienna, VA, US"
8,39067536,Concert,http://www.songkick.com/concerts/39067536-blac...,2019-11-13,1038.0,Black Cat,"Washington, DC, US"
9,39077211,Concert,http://www.songkick.com/concerts/39077211-nati...,2019-11-13,554416.0,"Concert Hall, Kennedy Center","Washington, DC, US"


In [35]:
artist = "blink 182"
resp = getArtistKey(artist)

NameError: name 'getArtistKey' is not defined